# Treinamento com interface de alto nível

## Importação das bibliotecas

In [1]:
# http://pytorch.org/
from os.path import exists

import torch

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## Criação da rede

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

## Treinamento

### Criando o objeto de treinamento

In [4]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Avaliação

In [6]:
# Verifica se existe um GPU disponível
use_cuda = torch.cuda.is_available()
use_cuda

True

In [7]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

epochs = 14
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316034
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.137710
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.659471
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.623563
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.284630
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.375001
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.379094
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.501750
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.406622
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.326084
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.417999
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.218462
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.197826
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.086519
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.071963
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.149118
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.136582
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.133690
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.380902
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.156134
Train Epoch: 1 [12800/60000 (